In [4]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2024-07-31 12:24:49--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.05s   

2024-07-31 12:24:50 (91.1 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [5]:
import numpy as np
import pandas as pd
import textwrap # This will help to wrap the text after tokenize
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [6]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
df = pd.read_csv('bbc_text_cls.csv')

In [8]:
#looking at our data columns and its values
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [9]:
#Types of labels we have
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [10]:
# Pick a label whose data we want to train from
label = 'tech'

In [11]:
texts = df[df['labels'] == label]['text']
texts.head()

1824    Ink helps drive democracy in Asia\n\nThe Kyrgy...
1825    China net cafe culture crackdown\n\nChinese au...
1826    Microsoft seeking spyware trojan\n\nMicrosoft ...
1827    Digital guru floats sub-$100 PC\n\nNicholas Ne...
1828    Technology gets the creative bug\n\nThe hi-tec...
Name: text, dtype: object

In [15]:
# collect counts
#This code will divide each sentance to each line,
#Each line act as a sentance and then it will break into each word
#Those words will assign to t_0(first word of sentace),t_1(second word of sentance),t_2 (3rd word of sentance) and we will try to change middle word with greatest probability word.
#How to check Greatest probability : so we will group by on tuple(1st and 3rd word) and (check the occurance of 2nd word / total words) gives you the probability
#So 2nd word with highest probability will replace the original 2nd word
probs = {} # key: (w(t-1), w(t+1)), value: {w(t): count(w(t))}

for doc in texts:
  lines = doc.split("\n")
  for line in lines:
    tokens = word_tokenize(line)
    for i in range(len(tokens) - 2):
      t_0 = tokens[i]
      t_1 = tokens[i + 1]
      t_2 = tokens[i + 2]
      key = (t_0, t_2)
      if key not in probs:
        probs[key] = {}

      # add count for middle token
      if t_1 not in probs[key]:
        probs[key][t_1] = 1
      else:
        probs[key][t_1] += 1

In [16]:
# normalize probabilities
for key, d in probs.items():
  # d should represent a distribution
  total = sum(d.values())
  for k, v in d.items():
    d[k] = v / total

In [17]:
prob

In [18]:
texts.iloc[0].split("\n")

['Ink helps drive democracy in Asia',
 '',
 "The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.",
 '',
 'This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections. The US government agreed to fund all expenses associated with this decision.',
 '',
 'The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies. The use of ink is only one part of a general effort to show commitment towards more open elections - 

In [19]:
T#This function will find the word to spin
def spin_document(doc):
  # split the document into lines (paragraphs)
  lines = doc.split("\n")
  output = []
  for line in lines:
    if line:
      new_line = spin_line(line)
    else:
      new_line = line
    output.append(new_line)
  return "\n".join(output)

In [20]:
detokenizer = TreebankWordDetokenizer()


In [21]:
texts.iloc[0].split("\n")[2]

"The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting."

In [22]:
detokenizer.detokenize(word_tokenize(texts.iloc[0].split("\n")[2]))


"The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting."

In [23]:
def sample_word(d):
  p0 = np.random.random()
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [28]:
#This line will replace the mid word based on max probability word
def spin_line(line):
  tokens = word_tokenize(line)
  i = 0
  output = [tokens[0]]
  while i < (len(tokens) - 2):
    t_0 = tokens[i]
    t_1 = tokens[i + 1]
    t_2 = tokens[i + 2]
    key = (t_0, t_2)
    p_dist = probs[key]
    if len(p_dist) > 1 and np.random.random() < 0.3:
      # let's replace the middle word
      middle = sample_word(p_dist)
      output.append(t_1)
      output.append("<" + middle + ">")
      output.append(t_2)

      # we won't replace the 3rd token since the middle
      # token was dependent on it
      # instead, skip ahead 2 steps
      i += 2
    else:
      # we won't replace this middle word
      output.append(t_1)
      i += 1
  # append the final token - only if there was no replacement
  if i == len(tokens) - 2:
    output.append(tokens[-1])
  return detokenizer.detokenize(output)

In [29]:
np.random.seed(1234)

In [26]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)

In [27]:
print(textwrap.fill(
    new_doc, replace_whitespace=False, fix_sentence_endings=True))

Format wars could 'confuse users'

Technology firms Sony, Philips,
Matsushita and Samsung are developing a common <new> way to stop
people pirating <with> digital music <music> and video.

The firms
want to make <make> a system that ensures files play on <on> the
hardware <ways> they make but also thwarts illegal copying . The move
could mean <need> more confusion for consumers already faced by many
different, and conflicting, content control systems <time>, experts
warned . They say there are no guarantees <doubt> the system will even
prevent piracy.

Currently many online stores wrap up downloadable
files in an own-brand control system that means they can <can> only be
played on a <a> small number <population> of media players . Systems
<But> that limit <limit> what people can do <bring> with the files
they download are known <not> as Digital Rights Management systems .
By setting up the alliance to work on <towards> a common <common>
control system, the firms said they hope to end t